In [1]:
import secrets
from sympy import isprime
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.primitives import padding
from cryptography.hazmat.backends import default_backend
import hashlib
import os

# -----------------------------
# Helper functions
# -----------------------------

def generate_prime(bits=16):
    """Generate a random prime number."""
    while True:
        n = secrets.randbits(bits)
        if n % 2 == 0:
            n += 1
        if isprime(n):
            return n

def derive_aes_key(shared_secret):
    """Derive a 16-byte AES key from the shared secret using SHA-256."""
    key_bytes = hashlib.sha256(str(shared_secret).encode()).digest()
    return key_bytes[:16]  # AES-128

def encrypt_aes(message, key):
    """Encrypt message using AES-CBC."""
    iv = os.urandom(16)
    padder = padding.PKCS7(128).padder()
    padded_data = padder.update(message.encode()) + padder.finalize()
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    encryptor = cipher.encryptor()
    ciphertext = encryptor.update(padded_data) + encryptor.finalize()
    return iv, ciphertext

def decrypt_aes(iv, ciphertext, key):
    """Decrypt AES-CBC ciphertext."""
    cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
    decryptor = cipher.decryptor()
    padded_plaintext = decryptor.update(ciphertext) + decryptor.finalize()
    unpadder = padding.PKCS7(128).unpadder()
    plaintext = unpadder.update(padded_plaintext) + unpadder.finalize()
    return plaintext.decode()

# -----------------------------
# Diffie-Hellman Key Exchange
# -----------------------------

# Step 1: Public parameters
p = generate_prime(bits=16)
g = secrets.randbelow(p - 2) + 2

print("===== Diffie-Hellman Parameters =====")
print("Prime (p):", p)
print("Generator (g):", g)

# Step 2: Private keys
a = secrets.randbelow(p - 2) + 1  # Alice
b = secrets.randbelow(p - 2) + 1  # Bob

print("\n===== Private Keys =====")
print("Alice's private key (a):", a)
print("Bob's private key (b):", b)

# Step 3: Public keys
A = pow(g, a, p)
B = pow(g, b, p)

print("\n===== Public Keys =====")
print("Alice's public key (A):", A)
print("Bob's public key (B):", B)

# Step 4: Shared secret
s_alice = pow(B, a, p)
s_bob = pow(A, b, p)

print("\n===== Shared Secret =====")
print("Shared secret computed by Alice:", s_alice)
print("Shared secret computed by Bob  :", s_bob)

# Step 5: Derive AES key from shared secret
aes_key = derive_aes_key(s_alice)
print("\nDerived AES Key (hex):", aes_key.hex())

# -----------------------------
# Encryption & Decryption
# -----------------------------
message = input("\nEnter the message to encrypt: ")

# Encrypt
iv, ciphertext = encrypt_aes(message, aes_key)
print("\n===== Encryption =====")
print("Initialization Vector (IV):", iv.hex())
print("Encrypted message (ciphertext in hex):", ciphertext.hex())

# Decrypt
decrypted_message = decrypt_aes(iv, ciphertext, aes_key)
print("\n===== Decryption =====")
print("Decrypted message:", decrypted_message)


===== Diffie-Hellman Parameters =====
Prime (p): 64157
Generator (g): 41725

===== Private Keys =====
Alice's private key (a): 61628
Bob's private key (b): 27514

===== Public Keys =====
Alice's public key (A): 1981
Bob's public key (B): 24311

===== Shared Secret =====
Shared secret computed by Alice: 4923
Shared secret computed by Bob  : 4923

Derived AES Key (hex): 66132b1c601d1f2c6696608d76e77c7c

===== Encryption =====
Initialization Vector (IV): b2eca5242bf5c73738221a8cbc31a7f5
Encrypted message (ciphertext in hex): 535b26bd9f0cbced52b8d44cb6421671

===== Decryption =====
Decrypted message: Hello world
